In [95]:
import pandas as pd
import datetime


def time_to_seconds(time_str):
    """Convert a time string in the format H:M:S.microseconds to seconds."""
    if not time_str:  # Handle empty strings
        return None
    if time_str != time_str:
        return None
    time_obj = datetime.datetime.strptime(time_str, "%H:%M:%S.%f")
    return time_obj.hour * 3600 + time_obj.minute * 60 + time_obj.second + time_obj.microsecond / 1_000_000

# Read res.csv
df = pd.read_csv('res.csv')
# All columns that have an int header
int_columns = [col for col in df.columns if col.isdigit()]
# Convert all columns to seconds
df[int_columns] = df.apply(lambda row: row[int_columns].apply(time_to_seconds), axis=1)

df

,metodo,8,16,32,64,1024,8192
0,RPF,221.729603,522.336412,1410.031252,4080.225128,NaN,NaN
1,Vanilla,5.053546,9.297575,19.009304,41.573534,NaN,NaN
2,Latest PBRTv2,1.800000,3.600000,7.200000,14.400000,232.2,1940.0


In [96]:
# Default plot
#import matplotlib.pyplot as plt
#df.plot()



# Advanced plot
import plotly.express as px
df_input = df
df_melted = df_input.melt(id_vars="metodo", var_name="n", value_name="time")
fig = px.line(
    df_melted,
    x="n",
    y="time",
    color="metodo",
    markers=True,
    labels={"size": "Size", "value": "Value", "metodo": "Method", "time": "Time (s)", "n": "Sample Per Pixel (spp)"},
    title="Comparación de tiempo de ejecución (PBRTv2)"
)

fig.update_layout(
  template="plotly_dark",
)

# Save image to ./imgs/analisis/execution_time.png
fig.write_image("./imgs/analisis/execution_time.png")

fig.show()

In [89]:
# Get list of all pngs in imgs/rpf
from typing import Union
import os
import glob
import numpy as np
from skimage.metrics import structural_similarity as ssim
pngs = glob.glob("./imgs/rpf/*.png")

# Load ground truth
import cv2
gt = cv2.imread("./imgs/rpf/Conference_8192_PBRTV2.png", cv2.IMREAD_COLOR)

# Score function
def image_scores(gt, img):
    s3 = np.mean((gt/255.0 - img/255.0) ** 2)
    s2 = ssim(gt, img, win_size=7, channel_axis=-1)  # Specify win_size and channel_axis
    #s1 = cv2.PSNR(gt, img)


    return s2, s3

# Capture the {experiment}_{spp}_{method}_....png
import re
data = []
data_header = ["png", "experiment", "spp", "method", "SSIM", "MSE"]
data.append(data_header)
for png in pngs:

    # Load image
    img = cv2.imread(png, cv2.IMREAD_COLOR)
    # Score image
    scores = image_scores(gt, img)

    match = re.search(r"(\w+)_(\d+)_(\w+)_", png)
    if  not match:
        # Special case of PBRTV2
        match = re.search(r"(\w+)_(\d+)_(\w+)", png)

    if match:
        experiment, spp_str, method = match.groups()
        spp = int(spp_str)

        this_data : list[Union[str, float]] = [png,]
        this_data.extend( [experiment, spp, method] )
        this_data.extend(scores)
        data.append(this_data)
# To DataFrame
df = pd.DataFrame(data[1:], columns=data[0])
# Rename PBRTV2 to Latest PBRTv2 in method column
df["method"] = df["method"].replace("PBRTV2", "Latest PBRTv2")
df

,png,experiment,spp,method,SSIM,MSE
0,./imgs/rpf\Conference_1024_PBRTV2.png,Conference,1024,Latest PBRTv2,0.664159,0.002329
1,./imgs/rpf\Conference_16_MC_0016.png,Conference,16,MC,0.383936,0.006742
2,./imgs/rpf\Conference_16_PBRTV2.png,Conference,16,Latest PBRTv2,0.379446,0.007624
3,./imgs/rpf\Conference_16_RPF_flt.png,Conference,16,RPF,0.832872,0.001755
4,./imgs/rpf\Conference_32_MC_0032.png,Conference,32,MC,0.468075,0.004067
5,./imgs/rpf\Conference_32_PBRTV2.png,Conference,32,Latest PBRTv2,0.438551,0.006772
6,./imgs/rpf\Conference_32_RPF_flt.png,Conference,32,RPF,0.826984,0.001943
7,./imgs/rpf\Conference_64_PBRTV2.png,Conference,64,Latest PBRTv2,0.492275,0.006282
8,./imgs/rpf\Conference_8192_PBRTV2.png,Conference,8192,Latest PBRTv2,1.000000,0.000000
9,./imgs/rpf\Conference_8_MC_0008.png,Conference,8,MC,0.315718,0.009857


In [92]:
df_input = df[["method", "spp", "SSIM"]]
# Only select method="RPF"
#df_input = df_input[df_input["method"] in 
# Apply log scale to spp
df_input["spp"] = np.log(df_input["spp"])
df_melted = df_input.melt(id_vars=["spp", "method"], var_name="metric", value_name="value")
df_melted = df_melted.sort_values(by=["spp", "method"])
fig = px.line(
    df_melted,
    x="spp",
    y="value",
    color="method",
    markers=True,
    labels={"size": "Size", "method": "Method", "value": "SSIM", "spp": "Log Samples Per Pixel ( log(spp) )"},
    title="Comparación de SSIM (PBRTv2)"
)
fig.update_layout(
  template="plotly_dark",
)
fig.write_image("./imgs/analisis/ssim.png")
fig.show()


C:\Users\Rodrigo\AppData\Local\Temp\ipykernel_34752\3280567603.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [93]:
df_input = df[["method", "spp", "MSE"]]
# Only select method="RPF"
#df_input = df_input[df_input["method"] in 
# Apply log scale to spp
df_input["spp"] = np.log(df_input["spp"])
df_melted = df_input.melt(id_vars=["spp", "method"], var_name="metric", value_name="value")
df_melted = df_melted.sort_values(by=["spp", "method"])
fig = px.line(
    df_melted,
    x="spp",
    y="value",
    color="method",
    markers=True,
    labels={"size": "Size", "method": "Method", "value": "MSE", "spp": "Log Samples Per Pixel ( log(spp) )"},
    title="Comparación de MSE (PBRTv2)"
)
fig.update_layout(
  template="plotly_dark",
)
fig.write_image("./imgs/analisis/mse.png")
fig.show()


C:\Users\Rodrigo\AppData\Local\Temp\ipykernel_34752\1837236948.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

